In [10]:
# Import Dependencies
import requests 
import json

# Developer API Key from spotify 
from config import skey

In [11]:
# Ensure key is inputted
skey

'f461a0c0267f42b9b4b4341bb5d8605c'

In [12]:
# Create a base URL
base_url = "https://api.spotify.com/v1/me/albums"

In [13]:
# Create Variable 
scan = requests.get(base_url)